In [ ]:
from google.colab import files
uploaded = files.upload()



Saving healthcare_updated.csv to healthcare_updated.csv


In [ ]:
import sqlite3
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("/content/healthcare_updated.csv")

# Create a connection to SQLite in-memory DB
conn = sqlite3.connect(":memory:")

# Save DataFrame to SQL table
df.to_sql("appointments", conn, index=False, if_exists="replace")


110526

### EDA

# 1. Total appointments and no shows

In [ ]:
query = """
SELECT
    COUNT(*) AS "Total Appointments",
    SUM(CASE WHEN "No-show" = '1' THEN 1 ELSE 0 END) AS Total_No_Shows,
    ROUND(SUM(CASE WHEN "No-show" = '1' THEN 1.0 ELSE 0 END) / COUNT(*) * 100, 2) AS No_Show_Percentage
FROM appointments;
"""
pd.read_sql_query(query, conn)


,Total Appointments,Total_No_Shows,No_Show_Percentage
0,110526,22319,20.19


### 2. Which gender has more No-shows ?

In [ ]:
query = """
SELECT
    Gender,
    COUNT(*) AS "Total Appointments",
    SUM(CASE WHEN "No-show" = '1' THEN 1 ELSE 0 END) AS Total_No_Shows,
    ROUND(SUM(CASE WHEN "No-show" = '1' THEN 1.0 ELSE 0 END) / COUNT(*) * 100, 2) AS No_Show_Percentage
FROM appointments
GROUP BY Gender
ORDER BY Total_No_Shows DESC;
"""
pd.read_sql_query(query, conn)

,Gender,Total Appointments,Total_No_Shows,No_Show_Percentage
0,F,71839,14594,20.31
1,M,38687,7725,19.97


### 3. What age groups are more likely to miss appointments ?

In [ ]:
query = """
SELECT
  AgeGroup,
  COUNT(*) AS Total_Appointments,
  SUM(CASE WHEN "No-show" = 1 THEN 1 ELSE 0 END) AS Total_No_Shows,
  ROUND(
    CAST(SUM(CASE WHEN "No-show" = 1 THEN 1.0 ELSE 0 END) AS FLOAT) * 100.0 / COUNT(*),
    2
  ) AS No_Show_Percentage
FROM (
  SELECT *,
    CASE
      WHEN Age BETWEEN 0 AND 9 THEN '0-9'
      WHEN Age BETWEEN 10 AND 19 THEN '10-19'
      WHEN Age BETWEEN 20 AND 29 THEN '20-29'
      WHEN Age BETWEEN 30 AND 39 THEN '30-39'
      WHEN Age BETWEEN 40 AND 49 THEN '40-49'
      WHEN Age BETWEEN 50 AND 59 THEN '50-59'
      WHEN Age BETWEEN 60 AND 69 THEN '60-69'
      WHEN Age BETWEEN 70 AND 79 THEN '70-79'
      ELSE '80+'
    END AS AgeGroup
  FROM appointments
)
GROUP BY AgeGroup
ORDER BY AgeGroup;
"""
pd.read_sql_query(query, conn)

,AgeGroup,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,0-9,17475,3484,19.94
1,10-19,12936,3257,25.18
2,20-29,13699,3380,24.67
3,30-39,15171,3300,21.75
4,40-49,14209,2880,20.27
5,50-59,15863,2776,17.50
6,60-69,11876,1790,15.07
7,70-79,5934,902,15.20
8,80+,3363,550,16.35


In [ ]:
df = pd.read_csv("/content/healthcare_updated.csv")

In [ ]:
df.head(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,waitingDays,AppointmentDayOfWeek,IsWeekend
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,-1,Friday,0
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,-1,Friday,0
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,-1,Friday,0
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,-1,Friday,0
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,-1,Friday,0


In [ ]:
df['Age'].unique()

array([ 62,  56,   8,  76,  23,  39,  21,  19,  30,  29,  22,  28,  54,
        15,  50,  40,  46,   4,  13,  65,  45,  51,  32,  12,  61,  38,
        79,  18,  63,  64,  85,  59,  55,  71,  49,  78,  31,  58,  27,
         6,   2,  11,   7,   0,   3,   1,  69,  68,  60,  67,  36,  10,
        35,  20,  26,  34,  33,  16,  42,   5,  47,  17,  41,  44,  37,
        24,  66,  77,  81,  70,  53,  75,  73,  52,  74,  43,  89,  57,
        14,   9,  48,  83,  72,  25,  80,  87,  88,  84,  82,  90,  94,
        86,  91,  98,  92,  96,  93,  95,  97, 102, 115, 100,  99])

In [ ]:
df.Age.min()

0

### 4. Does receiving an SMS impact attendance ?

In [ ]:
query = """
SELECT
   SMS_received,
   COUNT(*) AS "Total Appointments",  -- Added a comma here
   SUM(CASE WHEN "No-show" = '1' THEN 1 ELSE 0 END) AS Total_No_Shows,
   ROUND(SUM(CASE WHEN "No-show" = '1' THEN 1.0 ELSE 0 END) / COUNT(*) * 100, 2) AS No_Show_Percentage
FROM appointments
GROUP BY SMS_received;
"""
pd.read_sql_query(query, conn)

,SMS_received,Total Appointments,Total_No_Shows,No_Show_Percentage
0,0,75044,12535,16.70
1,1,35482,9784,27.57


### 5. Are no-shows more common on weekends or weekdays?

In [ ]:
query = """
SELECT
  IsWeekend,
  COUNT(*) AS Total_Appointments,
  SUM(CASE WHEN `No-show` = 1 THEN 1 ELSE 0 END) AS Total_No_Shows,
  ROUND(SUM(CASE WHEN `No-show` = 1 THEN 1.0 ELSE 0 END) * 100.0 / COUNT(*), 2) AS No_Show_Percentage
FROM appointments
GROUP BY IsWeekend;
"""
pd.read_sql_query(query, conn)

,IsWeekend,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,0,110487,22310,20.19
1,1,39,9,23.08


### 6. Do medical conditions like hypertension or diabetes affect no-show behavior?

In [ ]:
# Hypertension impact on no-shows
query1 = """
SELECT
  Hipertension,
  COUNT(*) AS Total_Appointments,
  SUM(`No-show`) AS Total_No_Shows,
  ROUND(SUM(`No-show`) * 100.0 / COUNT(*), 2) AS No_Show_Percentage
FROM appointments
GROUP BY Hipertension;
"""
result1 = pd.read_sql_query(query1, conn)
display(result1)

# Diabetes impact on no-shows
query2 = """
SELECT
  Diabetes,
  COUNT(*) AS Total_Appointments,
  SUM(`No-show`) AS Total_No_Shows,
  ROUND(SUM(`No-show`) * 100.0 / COUNT(*), 2) AS No_Show_Percentage
FROM appointments
GROUP BY Diabetes;
"""
result2 = pd.read_sql_query(query2, conn)
display(result2)

,Hipertension,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,0,88725,18547,20.9
1,1,21801,3772,17.3


,Diabetes,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,0,102583,20889,20.36
1,1,7943,1430,18.00


### 7. Which neighborhoods have the highest no-show rates?

In [ ]:
query = """
SELECT
  Neighbourhood,
  COUNT(*) AS Total_Appointments,
  SUM(CASE WHEN `No-show` = 1 THEN 1 ELSE 0 END) AS Total_No_Shows,
  ROUND(SUM(CASE WHEN `No-show` = 1 THEN 1.0 ELSE 0 END) * 100.0 / COUNT(*), 2) AS No_Show_Percentage
FROM appointments
GROUP BY Neighbourhood
ORDER BY No_Show_Percentage DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,Neighbourhood,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,ILHAS OCEÂNICAS DE TRINDADE,2,2,100.00
1,SANTOS DUMONT,1276,369,28.92
2,SANTA CECÍLIA,448,123,27.46
3,SANTA CLARA,506,134,26.48
4,ITARARÉ,3514,923,26.27
5,JESUS DE NAZARETH,2853,696,24.40
6,HORTO,175,42,24.00
7,ILHA DO PRÍNCIPE,2266,532,23.48
8,CARATOÍRA,2565,591,23.04
9,ANDORINHAS,2262,521,23.03


### 8. What is the distribution of waiting days, and how does it affect attendance?

In [ ]:
query = """
SELECT
  CASE
    WHEN waitingDays < 0 THEN 'Invalid'
    WHEN waitingDays BETWEEN 0 AND 3 THEN '0-3 days'
    WHEN waitingDays BETWEEN 4 AND 7 THEN '4-7 days'
    WHEN waitingDays BETWEEN 8 AND 14 THEN '8-14 days'
    WHEN waitingDays BETWEEN 15 AND 30 THEN '15-30 days'
    WHEN waitingDays BETWEEN 31 AND 60 THEN '31-60 days'
    WHEN waitingDays BETWEEN 61 AND 90 THEN '61-90 days'
    ELSE '90+ days'
  END AS Waiting_Day_Range,
  COUNT(*) AS Total_Appointments,
  SUM(CASE WHEN `No-show` = 1 THEN 1 ELSE 0 END) AS Total_No_Shows,
  ROUND(SUM(CASE WHEN `No-show` = 1 THEN 1.0 ELSE 0 END) * 100.0 / COUNT(*), 2) AS No_Show_Percentage
FROM appointments
GROUP BY Waiting_Day_Range
ORDER BY Total_Appointments DESC;
"""
pd.read_sql_query(query, conn)










,Waiting_Day_Range,Total_Appointments,Total_No_Shows,No_Show_Percentage
0,Invalid,38567,1797,4.66
1,0-3 days,19965,4590,22.99
2,15-30 days,16540,5379,32.52
3,4-7 days,14552,3852,26.47
4,8-14 days,11196,3496,31.23
5,31-60 days,7746,2640,34.08
6,61-90 days,1799,521,28.96
7,90+ days,161,44,27.33
